# Text Subject & Urdu Translation Assistant

This notebook helps you convert English text into an email-ready subject, a concise summary, and two Urdu renditions (everyday spoken and refined/formal) written in Perso‑Arabic script (اردو). It is designed to be reusable and easy to adapt for others in a public repository.

What this notebook produces:
- Suggested email subject (English)
- Short summary (English)
- Everyday / colloquial Urdu translations (Perso‑Arabic script)
- Refined / formal Urdu translations (Perso‑Arabic script)

Why use this:
- Quickly generate subject lines and human-readable summaries for long English passages.
- Produce Urdu outputs that use native script (not romanization), with both casual and formal variants.
- Useful for writers, localization, email automation, and language learning.

How it works (high level):
1. Paste or edit the English input in the `user_prompt` cell.
2. The notebook composes a system prompt that enforces output structure and Urdu script requirements.
3. It calls the OpenAI chat API to generate the outputs in Markdown.
4. The notebook post‑processes and renders the model response as formatted Markdown.

Quick setup:
- Ensure `OPENAI_API_KEY` is available in your environment (use a `.env` file or export it in your shell).
- Install dependencies from `requirements.txt` (e.g.   `pip install -r requirements.txt`).

Notes and best practices:
- Do NOT commit API keys to the repository; use environment variables or CI secrets.
- If the model returns romanized Urdu instead of Perso‑Arabic script, strengthen the system prompt to explicitly require Urdu script (this notebook already does this).
- You can adapt the prompts or model parameters to change tone, length, or formality.

Example input (already included in this notebook):
"Our children are not your children..." — run the next cell to see subject, summary, and Urdu variants.

In [ ]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
# Step 1: Create your prompts

system_prompt = """
You are a helpful assistant. Whenever the user requests Urdu, always RESPOND IN URDU USING THE PERSO-ARABIC SCRIPT (اردو). Do NOT use Latin letters or transliteration.

Output requirements:
- Use colloquial, everyday spoken Urdu (informal but polite) written in Perso-Arabic script.
- Do NOT romanize (do NOT return words like "salaam" — write "سلام" instead).
- Structure the response as Markdown with these sections (exact headings):
  1. Suggested subject
  2. Summary
  3. Original text
  4. Urdu translation (heading: "اردو:")
- First give complete response in english (suggested subject, summary, original text) 
followed by urdu every day spoken (suggested subject, summart, original text), 
followed by nice, refine, elite urdu (uggested subject, summart, original text). 
Urdu response should have headings in urdu too


Example enforcement: If asked to translate "Hello" -> output: "سلام" (NOT "salaam").

Respond ONLY with the Markdown (no surrounding code fences). Use blank lines between paragraphs so the Markdown displays correctly.
"""

user_prompt = """
our children are not your children. They are the sons and daughters of Life's longing for itself. 
They come through you but not from you, And though they are with you yet they belong not to you.

You may give them your love but not your thoughts, For they have their own thoughts. 
You may house their bodies but not their souls, For their souls dwell in the house of tomorrow, which you cannot visit, not even in your dreams. You may strive to be like them, but seek not to make them like you.

For life goes not backward nor tarries with yesterday.

You are the bows from which your children as living arrows are sent forth. 
The archer sees the mark upon the path of the infinite, and He bends you with His might that His arrows may go swift and far. 
Let your bending in the archer's hand be for gladness; For even as He loves the arrow that flies, so He loves also the bow that is stable.
"""

In [ ]:
# Step 2: Make the messages list
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]



# Step 3: Call OpenAI
# instantiate the client with the API key if available (new-style client)
client = OpenAI(api_key=api_key) if api_key else OpenAI()
response = client.chat.completions.create(model="gpt-4.1-nano", messages=messages)
content = response.choices[0].message.content

# Step 4: normalize escaped sequences and display as Markdown
print('repr(content):', repr(content))
# If the model returned escaped unicode (e.g. '\u0627') or literal backslash-n sequences, decode/unescape them
try:
    # First try to decode unicode escape sequences (\uXXXX -> actual characters)
    if '\\u' in content or '\\U' in content:
        content = bytes(content, 'utf-8').decode('unicode_escape')
    # Then normalize literal backslash-n sequences to real newlines
    content = content.replace('\\\\n', '\\n').replace('\\n', '\n').replace('\\\n', '\n')
except Exception as e:
    # If something goes wrong, at least replace common escaped newlines
    content = content.replace('\\\\n', '\\n').replace('\\n', '\n')

display(Markdown(content))